In [1]:
library(feather)
library(varbvs)

In [2]:
get_rds_toy <- function(datalist){
    for (dat in datalist){
        newdat = read_feather(dat)
        newmatrix = matrix(as.double(unlist(newdat)), dim(newdat)[1], dim(newdat)[2])
        colnames(newmatrix) = colnames(newdat)
        newfit = varbvs(newmatrix[,-1], NULL, newmatrix[,1], family = "binomial", 
                        logodds = seq(-3.5,-1,0.1), sa = 1, verbose = F)
        newfit$summary = summary(newfit)
        newfit$w = normalizelogweights(newfit$logw)
        newfit$pip = newfit$alpha %*% c(newfit$w)
        filename = paste(dat, ".RDS", sep="")
        saveRDS(newfit, filename)
        return (newfit$summary$top.vars)
    }
}

In [5]:
# get_rds_toy(c("data/toy_n000_600_n100_100_n010_100_n001_100_n110_100_n011_100_n111_100.feather"))
get_rds_toy(c("data/toy_pos15_causal8_n1250_const0.5.feather"))

index,variable,prob,PVE,coef,Pr(coef.>0.95)
8,pos8,0.99999301,NA,0.7001739,"[+0.483,+0.920]"
4,pos4,0.01508044,NA,0.1732682,"[-0.068,+0.401]"
3,pos3,0.01411433,NA,0.1681157,"[-0.070,+0.405]"
14,pos14,0.01108493,NA,-0.1472181,"[-0.373,+0.068]"
9,pos9,0.01067776,NA,-0.1419294,"[-0.373,+0.085]"


In [4]:
newdat = read_feather("data/toy_pos15_causal8_n1250.feather")
newmatrix = matrix(as.double(unlist(newdat)), dim(newdat)[1], dim(newdat)[2])
colnames(newmatrix) = colnames(newdat)
# print (dim(newdat)[1])
newfit = varbvs(newmatrix[,-1], NULL, newmatrix[,1], family = "binomial", 
                logodds = seq(-3.5,-1,0.1), sa = 1, verbose = F)
newfit$summary = summary(newfit)
newfit$w = normalizelogweights(newfit$logw)
newfit$pip = newfit$alpha %*% c(newfit$w)
newfit$beta = newfit$mu %*% c(newfit$w)
newfit$var_names = colnames(newdat[-1])
print (summary(newfit))
dimnames(newfit$pip)[[1]]

Summary of fitted Bayesian variable selection model:
family:     binomial   num. hyperparameter settings: 26
samples:    1250       iid variable selection prior: yes
variables:  15         fit prior var. of coefs (sa): no
covariates: 1          fit approx. factors (eta):    yes
maximum log-likelihood lower bound: -786.2700
Hyperparameters: 
        estimate Pr>0.95             candidate values
sa            NA [NA,NA]             1--1
logodds    -1.53 [-2.50,-1.00]       (-3.50)--(-1.00)
Selected variables by probability cutoff:
>0.10 >0.25 >0.50 >0.75 >0.90 >0.95 
    1     1     1     1     1     1 
Top 5 variables by inclusion probability:
  index variable    prob PVE coef*  Pr(coef.>0.95)
1     8     pos8 1.00000  NA 1.596 [+1.366,+1.821]
2    13    pos13 0.00816  NA 0.115 [-0.101,+0.355]
3    10    pos10 0.00802  NA 0.113 [-0.105,+0.357]
4    11    pos11 0.00775  NA 0.109 [-0.111,+0.318]
5    12    pos12 0.00752  NA 0.105 [-0.127,+0.335]
*See help(varbvs) about interpreting coeffi

[1] "pos1"  "pos2"  "pos3"  "pos4"  "pos5"  "pos6"  "pos7"  "pos8"  "pos9" 
[10] "pos10" "pos11" "pos12" "pos13" "pos14" "pos15"